In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [23]:
! rm -r sketch2ppt
! git clone https://github.com/fateh288/sketch2ppt.git



Cloning into 'sketch2ppt'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 125 (delta 63), reused 96 (delta 40), pack-reused 0
Receiving objects: 100% (125/125), 3.55 MiB | 6.20 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [24]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
import h5py
def load_data():
    f = h5py.File('sketch2ppt/hdf5/shapes_classification.hdf5', 'r')
    return (f['train_img'][...],f['train_labels'][...]),(f['test_img'][...],f['test_labels'][...])

In [26]:
(train_images,train_labels),(test_images,test_labels) = load_data()
#train_images, test_images = train_images / 255.0, test_images / 255.0
print(train_images.shape, type(train_images))
print(train_labels.shape, type(train_labels))

(804, 128, 128, 1) <class 'numpy.ndarray'>
(804,) <class 'numpy.ndarray'>


In [28]:
print(np.min(train_labels))
print(np.max(train_labels))

0
3


In [39]:
IMG_SIZE = 128

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.8),
])
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

model = models.Sequential()
model.add(resize_and_rescale)
# # model.add(data_augmentation)
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.Conv2D(512, (3, 3), activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.Conv2D(512, (3, 3), activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
# model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

In [40]:
custom_adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=custom_adam,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=1000,
                    validation_data=(test_images, test_labels))


Epoch 1/500
26/26 [==============================] - 1s 25ms/step - loss: 1.3987 - accuracy: 0.2811 - val_loss: 1.3531 - val_accuracy: 0.3682
Epoch 2/500
26/26 [==============================] - 0s 19ms/step - loss: 1.2934 - accuracy: 0.4080 - val_loss: 1.2302 - val_accuracy: 0.5672
Epoch 3/500
26/26 [==============================] - 0s 19ms/step - loss: 1.2342 - accuracy: 0.4577 - val_loss: 1.1160 - val_accuracy: 0.6766
Epoch 4/500
26/26 [==============================] - 0s 19ms/step - loss: 1.1595 - accuracy: 0.5522 - val_loss: 1.0732 - val_accuracy: 0.7114
Epoch 5/500
26/26 [==============================] - 0s 19ms/step - loss: 1.1472 - accuracy: 0.5958 - val_loss: 1.0492 - val_accuracy: 0.7264
Epoch 6/500
26/26 [==============================] - 0s 19ms/step - loss: 1.1276 - accuracy: 0.6256 - val_loss: 1.0841 - val_accuracy: 0.6418
Epoch 7/500
26/26 [==============================] - 0s 19ms/step - loss: 1.1157 - accuracy: 0.6244 - val_loss: 1.0091 - val_accuracy: 0.7562
Epoch 

In [41]:
model.save('/content/gdrive/My Drive/saved_model_sketch2ppt/model_center_conv_fully_500')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/saved_model_sketch2ppt/model_center_conv_fully_500/assets
